# CTD Module

This module is the only module to ingest the CTD data files from the cruise. It is currently setup for a CTD downcast file and a CTD Log file but is easily adaptable for a single CTD data file. This module creates rdata structures for use with the other scripts in this suite.

In [ ]:
source('source.r')

In [ ]:
## Downcast file
ctd = read.xlsx('Raw Data/CTD/DataZoo - P1706 CTD Downcast Data.xlsx')
ctd$Time = conv_excel_time(ctd$DateTime, tz = 'UTC')

## CTD Log file
ctd.log = read.xlsx('Raw Data/CTD/DataZoo - P1706 CTD Cast Log.xlsx', sheet='Data')
ctd.log$time = conv_excel_time(ctd.log$DateTime, tz = 'GMT')

## Check file structures and variaible types
str(ctd)
str(ctd.log)

In [ ]:
## A function to take the log file position and add it to the downcast data.
add.position = function(ctd, ctd.log) {
    ctd$Lat = NA
    ctd$Lon = NA
    ctd$Type = NA
    
    rej = c()
    for (i in 1:nrow(ctd)) {
        l = which(ctd.log$EventNo == ctd$EventNo[i])

        if (length(l) == 1) {
            ctd$Lat[i] = ctd.log$Lat[l]
            ctd$Lon[i] = ctd.log$Lon[l]
            ctd$Type = ctd.log$Station.Type[l]
        } else {
            rej = c(rej, i)
        }
    }
    print(paste0('Number of unmatched bins: ', length(rej)))
    ctd
}

ctd = add.position(ctd, ctd.log)

In [ ]:
save(ctd, file = 'RStates/CTD.all.rdata')

---

# Par and Transmission analysis

In [ ]:
pt.comparison.plot = function(cast){
    l = which(ctd$Cast == cast)
    
    if (max(ctd$PAR[l], na.rm = TRUE) < 100) {
        return()
    }
    PAR = ctd$PAR[l] / ctd$PAR[l[1]]
    TRANS = cumprod(ctd$Trans[l]/100)
    Depth = ctd$Pressure[l]
    
    plot(PAR, Depth, ylim=c(200, 0), yaxs='i', main=paste('Cast -', cast), pch=16, cex=0.6)
    points(TRANS, Depth, col='blue', pch=16, cex=0.3)
    
    plot(PAR, TRANS^2)
    lm.log = lm(TRANS^2 ~ PAR)
    lines(PAR, predict(lm.log)[1:length(PAR)])
}

unique(ctd$Cast)

In [ ]:
pdf('Output/CTD - Transmission and PAR analysis.pdf')
par(mfrow=c(2,2))

for (i in unique(ctd$Cast)) {
    pt.comparison.plot(i)
}

dev.off()

In [ ]:
tchl.comparison.plot = function(cast){
    l = which(ctd$Cast == cast & ctd$Pressure < 100)
    
    if (max(ctd$Fluor[l], na.rm = TRUE) < 1) {
        return()
    }
    
    CHL = ctd$Fluor[l] / max(ctd$Fluor[l], na.rm = TRUE)
    TRANS = ctd$Trans[l]/100
    Depth = ctd$Pressure[l]
    
    plot(CHL, Depth, ylim=c(200, 0), yaxs='i', main=paste('Cast -', cast), pch=16, cex=0.6, xlim=c(0,2))
    points(TRANS, Depth, col='blue', pch=16, cex=0.3)
    
    plot(CHL, TRANS, col=make.div.pal(ctd$Pressure[l], n = 100), pch=16, ylim=c(0.8,1), xlim=c(0,1))
    lm.log = lm(TRANS ~ CHL)
    points(CHL, predict(lm.log)[1:length(CHL)], pch=16, cex=0.5, col='red')
}

unique(ctd$Cast)

In [ ]:
pdf('Output/CTD - Transmission and CHL analysis.pdf')

par(mfrow=c(2,2))

for (i in unique(ctd$Cast)) {
    try(tchl.comparison.plot(i))
}

dev.off()

## Plot Transmission Profiles

In [ ]:
par(mfrow=c(3,3))
for (i in unique(ctd$Cast)) {
    l = which(ctd$Cast == i)
    plot(ctd$Trans[l], ctd$Depth[l], ylim=c(100,0), yaxs='i', xlab='Transmission', xlim=c(80,100), main=i, pch=16, cex=0.4)
}